# loading order data from csv using pyspark

In [1]:
from google.colab import files
uploaded=files.upload()

Saving cleaned_suppliers.csv to cleaned_suppliers.csv
Saving cleaned_inventory.csv to cleaned_inventory.csv
Saving cleaned_orders.csv to cleaned_orders.csv


In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("supply_chain_monitor_optimization").getOrCreate()
spark

In [4]:
sf=spark.read.csv('cleaned_suppliers.csv',header=True,inferSchema=True)
of=spark.read.csv('cleaned_orders.csv',header=True,inferSchema=True)
nf=spark.read.csv('cleaned_inventory.csv',header=True,inferSchema=True)
sf.show()
of.show()
nf.show()

+-----------+----------------+--------------------+
|supplier_id|            name|       contact_email|
+-----------+----------------+--------------------+
|          1|Alpha Components|alpha@components.com|
|          2| Beta Industrial| beta@industrial.com|
|          3|    Catalyst Co.| contact@catalyst.co|
|          4|  Delta Supplies|info@deltasupplie...|
|          5|   Epsilon Goods|   epsilon@goods.net|
+-----------+----------------+--------------------+

+--------+-----------+----------+-------------+--------+---------+------------+
|order_id|supplier_id|order_date|delivery_date|quantity|   status|delayed_days|
+--------+-----------+----------+-------------+--------+---------+------------+
|       1|          1|2025-07-01|   2025-07-08|     100|  pending|           7|
|       2|          2|2025-07-02|   2025-07-06|     200|fulfilled|           0|
|       3|          3|2025-07-03|   2025-07-03|     150|cancelled|           0|
|       4|          1|2025-07-05|   2025-07-08|    

# Filter delayed shipments

In [6]:
from pyspark.sql.functions import col
of_filtered=of.filter(col('status')=='pending')
of_filtered.show()

+--------+-----------+----------+-------------+--------+-------+------------+
|order_id|supplier_id|order_date|delivery_date|quantity| status|delayed_days|
+--------+-----------+----------+-------------+--------+-------+------------+
|       1|          1|2025-07-01|   2025-07-08|     100|pending|           7|
|       5|          4|2025-07-07|   2025-07-14|      75|pending|           7|
|       8|          3|2025-07-11|   2025-07-18|      50|pending|           7|
|      11|          1|2025-07-14|   2025-07-21|      90|pending|           7|
|      14|          4|2025-07-17|   2025-07-24|      80|pending|           7|
+--------+-----------+----------+-------------+--------+-------+------------+



# Group by supplier and count delayed orders

In [11]:
grouped_suppliers=of_filtered.groupBy('supplier_id').count()
grouped_suppliers.show()

+-----------+-----+
|supplier_id|count|
+-----------+-----+
|          1|    2|
|          3|    1|
|          4|    2|
+-----------+-----+



# saving the the processed data as acsv file


In [12]:
of_filtered.write.csv('filtered_orders.csv',header=True)
grouped_suppliers.write.csv('grouped_by_supplier_delay.csv',header=True)